In [1]:
# Workspace problem with several narrow gaps

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D
import os
import csv
from random import randint, random
import time

# (restrict tensorflow memory growth)
os.environ["CUDA_VISIBLE_DEVICES"]="1"
config = tf.ConfigProto()
config.gpu_options.allow_growth=True

In [2]:
# neural network parameters
mb_size = 256
h_Q_dim = 512
h_P_dim = 512

c = 0
# learning rate
lr = 1e-4

# problem dimensions
dim = 7
dataElements = dim*3 + 16 # sample (7D), init (7D), goal (7D), cond (16 points) //total = 37

z_dim = 3 # latent
X_dim = dim # samples
y_dim = dim # reconstruction of the original point
c_dim = dataElements - dim # dimension of conditioning variable

In [3]:
# read in data from csv
filename = 'samplingdata.txt'
f = open(filename, 'rb')
reader = csv.reader(f, delimiter=',')
count = 0
data_list = []
for row in reader:
    data_list.append(map(float,row[0:dataElements]))

data = np.array(data_list,dtype='d')

######################### Comment it #############
# data[:,3:6] = 0
##################################################
print("shape of array: ",data.shape)

numEntries = data.shape[0]

('shape of array: ', (3997, 37))


In [4]:
# split the inputs and conditions into test train (to be processed in the next step into an occupancy grid representation)
ratioTestTrain = 0.8;
numTrain = int(numEntries*ratioTestTrain)

X_train = data[0:numTrain,0:dim] # state: x, y, z, xdot, ydot, zdot
c_train = data[0:numTrain,dim:dataElements] # conditions: gaps, init (6), goal (6)

X_test = data[numTrain:numEntries,0:dim]
c_test = data[numTrain:numEntries,dim:dataElements]

#########################################################
c_train1 = []
c_test1 = []
c_train1 = c_train
c_test1 = c_test
#########################################################
numTest = X_test.shape[0]

In [5]:
# print("shape of final obstacle = ",obs.shape)
print("shape of c_train1 = ", c_train1.shape)
print("shape of c_test1 = ",c_test1.shape)

('shape of c_train1 = ', (3197, 30))
('shape of c_test1 = ', (800, 30))


In [6]:
# define networks
X = tf.placeholder(tf.float32, shape=[None, X_dim])
c = tf.placeholder(tf.float32, shape=[None, 30])
    
# Q
inputs_Q = tf.concat(axis=1, values=[X,c])

dense_Q1 = tf.layers.dense(inputs=inputs_Q, units=h_Q_dim, activation=tf.nn.relu)
dropout_Q1 = tf.layers.dropout(inputs=dense_Q1, rate=0.5)
dense_Q2 = tf.layers.dense(inputs=dropout_Q1, units=h_Q_dim, activation=tf.nn.relu)

z_mu = tf.layers.dense(inputs=dense_Q2, units=z_dim) # output here is z_mu
z_logvar = tf.layers.dense(inputs=dense_Q2, units=z_dim) # output here is z_logvar

# P
eps = tf.random_normal(shape=tf.shape(z_mu))
z = z_mu + tf.exp(z_logvar / 2) * eps
inputs_P = tf.concat(axis=1, values=[z,c])

dense_P1 = tf.layers.dense(inputs=inputs_P, units=h_P_dim, activation=tf.nn.relu)
dropout_P1 = tf.layers.dropout(inputs=dense_P1, rate=0.5)
dense_P2 = tf.layers.dense(inputs=dropout_P1, units=h_P_dim, activation=tf.nn.relu)

y = tf.layers.dense(inputs=dense_P2, units=X_dim) # fix to also output y

# training
########### comment in the one with 0 weight and uncomment the other ###########
w = [[1, 1, 1, 1, 1, 1, 1]];
# w = [[1, 1, 1, 0, 0, 0]];
recon_loss = tf.losses.mean_squared_error(labels=X, predictions=y, weights=w)

# TODO: fix loss function for angles going around
kl_loss = 10**-4 * 2 * tf.reduce_sum(tf.exp(z_logvar) + z_mu**2 - 1. - z_logvar, 1)

cvae_loss = tf.reduce_mean(kl_loss + recon_loss)

train_step = tf.train.AdamOptimizer(lr).minimize(cvae_loss)

sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())
it = 0;

In [9]:
saver = tf.train.Saver()
path_ = os.getcwd() + "/checkpoints/model.ckpt"
print("path = ",path_)
print("numTrain = ",numTrain)
try:
    saver.restore(sess, path_)
    print("Model Restored!!")
except Exception as e:
    print("Could not restore checkpoint!")
    print(e)
x1 = []
y1 = []    
for it in range(it,it+500001):
    # randomly generate batches
    batch_elements = [randint(0,numTrain-1) for n in range(0,mb_size)]
    X_mb = X_train[batch_elements,:]
    c_mb = c_train1[batch_elements,:]

    _, loss = sess.run([train_step, cvae_loss], feed_dict={X: X_mb, c: c_mb})

    if it % 1000 == 0:
        print('Iter: {}'.format(it))
        print('Loss: {:.4}'. format(loss))
        x1.append(it)
        y1.append(loss)
        saver.save(sess, path_)
        print()

('path = ', '/home/vernwalrahul/projects/LearningRoadmaps/cvae_learner/checkpoints/model.ckpt')
('numTrain = ', 3197)
INFO:tensorflow:Restoring parameters from /home/vernwalrahul/projects/LearningRoadmaps/cvae_learner/checkpoints/model.ckpt
Model Restored!!
Iter: 19000
Loss: 0.05586
()
Iter: 20000
Loss: 0.0531
()
Iter: 21000
Loss: 0.04973
()
Iter: 22000
Loss: 0.04976
()
Iter: 23000
Loss: 0.04503
()
Iter: 24000
Loss: 0.05031
()
Iter: 25000
Loss: 0.04189
()
Iter: 26000
Loss: 0.04031
()
Iter: 27000
Loss: 0.03769
()
Iter: 28000
Loss: 0.0335
()
Iter: 29000
Loss: 0.03664
()
Iter: 30000
Loss: 0.03592
()
Iter: 31000
Loss: 0.03331
()
Iter: 32000
Loss: 0.03543
()
Iter: 33000
Loss: 0.03587
()
Iter: 34000
Loss: 0.02963
()
Iter: 35000
Loss: 0.03207
()
Iter: 36000
Loss: 0.02793
()
Iter: 37000
Loss: 0.02654
()
Iter: 38000
Loss: 0.02582
()
Iter: 39000
Loss: 0.02382
()
Iter: 40000
Loss: 0.02365
()
Iter: 41000
Loss: 0.02276
()
Iter: 42000
Loss: 0.0235
()
Iter: 43000
Loss: 0.02111
()
Iter: 44000
Loss: 0.

KeyboardInterrupt: 